In [ ]:
import warnings
warnings.simplefilter(action='ignore',category=FutureWarning)
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
churn_df=pd.read_csv(r'/content/drive/MyDrive/churn.csv')
churn_df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


droping features to have a simplified version of dataset 

In [ ]:
churn_df=churn_df.drop(columns=['PaymentMethod','Partner','Dependents','PhoneService','MultipleLines','InternetService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies'])
churn_df.head()

,customerID,gender,SeniorCitizen,tenure,Contract,PaperlessBilling,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,1,Month-to-month,Yes,29.85,29.85,No
1,5575-GNVDE,Male,0,34,One year,No,56.95,1889.5,No
2,3668-QPYBK,Male,0,2,Month-to-month,Yes,53.85,108.15,Yes
3,7795-CFOCW,Male,0,45,One year,No,42.30,1840.75,No
4,9237-HQITU,Female,0,2,Month-to-month,Yes,70.70,151.65,Yes


In [ ]:
churn_df_master=churn_df.copy()

In [ ]:
churn_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   tenure            7043 non-null   int64  
 4   Contract          7043 non-null   object 
 5   PaperlessBilling  7043 non-null   object 
 6   MonthlyCharges    7043 non-null   float64
 7   TotalCharges      7043 non-null   object 
 8   Churn             7043 non-null   object 
dtypes: float64(1), int64(2), object(6)
memory usage: 495.3+ KB


In [ ]:
churn_df.tenure.unique()

array([ 1, 34,  2, 45,  8, 22, 10, 28, 62, 13, 16, 58, 49, 25, 69, 52, 71,
       21, 12, 30, 47, 72, 17, 27,  5, 46, 11, 70, 63, 43, 15, 60, 18, 66,
        9,  3, 31, 50, 64, 56,  7, 42, 35, 48, 29, 65, 38, 68, 32, 55, 37,
       36, 41,  6,  4, 33, 67, 23, 57, 61, 14, 20, 53, 40, 59, 24, 44, 19,
       54, 51, 26,  0, 39])

In [ ]:
#churn_df=churn_df.replace(to_replace={'tenure':{'One':'1','Four':'4'}},value=None)

In [ ]:
#churn_df.tenure.unique()

In [ ]:
#churn_df=churn_df.astype({'tenure':'int64'})

In [ ]:
churn_df.SeniorCitizen.unique()

array([0, 1])

In [ ]:
#churn_df['SeniorCitizen']=churn_df['SeniorCitizen'].fillna(0)

In [ ]:
churn_df.isna().sum()

customerID          0
gender              0
SeniorCitizen       0
tenure              0
Contract            0
PaperlessBilling    0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [ ]:
#churn_df['TotalCharges']=churn_df['TotalCharges'].fillna(0)

rows with missing value

In [ ]:
churn_df[churn_df.isna().any(axis=1)]

In [ ]:
churn_df['TotalCharges'].describe()

In [ ]:
churn_df[churn_df['Churn']=='No'].describe()

refer to the min and max values in tenure and total charges. Significant difference. So lets do a box plot

In [ ]:
plt.figure(figsize=(15,5))
sns.boxplot(x='TotalCharges',y='Churn',data=churn_df)

Churn Yes has outliers
Churn No is not normal distributon. May be mean or median is not a best way to impute

In [ ]:
sns.distplot(churn_df['TotalCharges'],hist=True)

right skewed data. Try squareroot,log etc function to seeif it can be transfrormed


In [ ]:
sns.distplot(churn_df['TotalCharges'].apply(np.sqrt),hist=True)

It is a better distribution

Monthly charges and Total charges are dependent and lets explore if it is useful to impute 

In [ ]:
sns.lmplot(x='MonthlyCharges',y='TotalCharges',data=churn_df,fit_reg=True, hue="Churn", markers=["o", "x"], palette="Blues_d") 

the above plot represents both yes and no churn and the line fit still has errors. Let us split yes and no churn sepretaley and view it

In [ ]:
sns.lmplot(x='MonthlyCharges',y='TotalCharges',data=churn_df,fit_reg=True, col="Churn", hue="Churn", line_kws={'color':'red'}) 

In [ ]:
churn_df[['TotalCharges','MonthlyCharges']].corr()

check correlation only for No churn

In [ ]:
churn_df[churn_df['Churn']=='No'][['TotalCharges','MonthlyCharges']].corr()

Now there is an improved correlation. Now that we have established there is a correlation lets create a data frame with no no to build a model



In [ ]:
churn_no_df=churn_df[(churn_df['Churn']=='No')&(churn_df['TotalCharges'].isna()==False)]
churn_no_df.shape[0]

In [ ]:
churn_no_df.isna().any()

In [ ]:
from scipy import stats
slope, intercept, r_value, p_value, std_err=stats.linregress(churn_no_df['MonthlyCharges'],churn_no_df['TotalCharges'])

In [ ]:
print ("slope     : " ,slope)
print ("intercept  : " ,intercept)
print ("r_value :  " ,r_value)
print ("p_value :  ", p_value)
print ("std_err :  ",std_err)
print("R-squared: ",  r_value**2)